In [1]:
import os
import theano
from theano import *
import theano.tensor as tt
from theano.compile.ops import as_op
import matplotlib.pyplot as plt
from argparse import Namespace
import pandas as pd
import numpy as np
import pymc3 as pm
import argparse
import pickle
import json
import math
import sys  

sys.path.insert(0, '/Users/Yannis/code/fibe2-mini-project/models')
from LinearReservoirModel import LinearReservoirModel as LRM

# Import data

In [ ]:
# Get current working directory and project root directory
cwd = os.getcwd()
rd = os.path.join(cwd.split('fibe2-mini-project/', 1)[0])
if not rd.endswith('fibe2-mini-project'):
    rd = os.path.join(cwd.split('fibe2-mini-project/', 1)[0],'fibe2-mini-project')

In [ ]:
model0data = pd.read_csv(os.path.join(rd,'data','output','simulations','linear_reservoir_simulation.csv'))
model1data = pd.read_csv(os.path.join(rd,'data','output','simulations','nonlinear_reservoir_simulation.csv'))
model2data = pd.read_csv(os.path.join(rd,'data','output','simulations','hymod_simulation.csv'))

# Store net net_rainfall
nr = model0data['net_rainfall'].values.tolist()
n = len(nr)


with open(os.path.join(rd,'data','output','simulations/linear_reservoir_simulation_true_parameters.json'), 'r') as f:
    lrm_true_params = json.load(f)
lrm_true_args = Namespace(**lrm_true_params)

with open(os.path.join(rd,'data','output','simulations/nonlinear_reservoir_simulation_true_parameters.json'), 'r') as f:
    nlrm_true_params = json.load(f)
nlrm_true_args = Namespace(**nlrm_true_params)

with open(os.path.join(rd,'data','output','simulations/hymod_simulation_true_parameters.json'), 'r') as f:
    hymod_true_params = json.load(f)
hymod_true_args = Namespace(**hymod_true_params)

In [ ]:
lrm = LRM(nr,lrm_true_args)

@as_op(itypes=[tt.dscalar], otypes=[tt.dmatrix])
def th_forward_model(param1):
    parameter_list = [param1]

    th_states = lrm.simulate(parameter_list,lrm_true_args.fatconv)
    return th_states

In [ ]:
# Path to files
csv_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples.csv')

LRMtrace_LRMdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_LRMdata_trace.pickle')
LRMmodel_LRMdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_LRMdata_model.pickle')

LRMtrace_NLRMdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_NLRMdata_trace.pickle')
LRMmodel_NLRMdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_NLRMdata_model.pickle')

In [ ]:
# Read files
results = pd.read_csv(csv_file)

LRMtrace_LRMdata = open(LRMtrace_LRMdata_file,"rb")
LRMmodel_LRMdata = open(LRMmodel_LRMdata_file,"rb")
LRMtrace_LRMdata = pickle.load(LRMtrace_LRMdata)
LRMmodel_LRMdata = pickle.load(LRMmodel_LRMdata)


LRMtrace_NLRMdata = open(LRMtrace_NLRMdata_file,"rb")
LRMmodel_NLRMdata = open(LRMmodel_NLRMdata_file,"rb")
LRMtrace_NLRMdata = pickle.load(LRMtrace_NLRMdata)
LRMmodel_NLRMdata = pickle.load(LRMmodel_NLRMdata)

traces = {"LRM":LRMtrace_LRMdata,"NLRM":LRMtrace_NLRMdata}
models = {"LRM":LRMmodel_LRMdata,"NLRM":LRMmodel_NLRMdata}

In [ ]:
keys = ['current_model','true_model','parameter','marginal_likelihood','mean', 'sd', 'mc_error', 'hpd_2.5', 'hpd_97.5']
results = pd.DataFrame(columns=keys)
for mi in ['LRM','NLRM']:#,'HYMOD']:
    vals = np.append(np.array(['LRM',mi,'k',models[mi].marginal_likelihood]),pm.summary(traces[mi], ['k']).values[0])
    results = results.append(dict(zip(keys, vals)),ignore_index=True)
    vals = np.append(np.array(['LRM',mi,'sigma',models[mi].marginal_likelihood]),pm.summary(traces[mi], ['sigma']).values[0])
    results = results.append(dict(zip(keys, vals)),ignore_index=True)

In [ ]:
results

In [ ]:
lrm_true_params

In [ ]:
nlrm_true_params

In [ ]:
hymod_true_params

In [ ]:
# Choose number of posterior samples
npostsamples = 1000

In [ ]:
_, ax = plt.subplots(figsize=(9, 6))
ppc_0 = pm.sample_posterior_predictive(LRMtrace_LRMdata, npostsamples, LRMmodel_LRMdata, size=(n, 20))
ppc_1 = pm.sample_posterior_predictive(LRMtrace_NLRMdata, npostsamples, LRMmodel_NLRMdata, size=(n, 20))
for m_0, m_1 in zip(ppc_0['Q_obs'].T, ppc_1['Q_obs'].T):
    pm.kdeplot(np.mean(m_0, 0), ax=ax, plot_kwargs={'color':'C0'})
    pm.kdeplot(np.mean(m_1, 0), ax=ax, plot_kwargs={'color':'C1'})
ax.plot([], label='LRM model on LRM data')
ax.plot([], label='LRM model on NLRM data')
ax.legend(fontsize=14)
ax.set_xlabel(u'Q', fontsize=14)
ax.set_yticks([]);

# Plot traces

In [ ]:
# _, ax = plt.subplots(figsize=(9, 6))
# ppc_0 = pm.sample_posterior_predictive(traces[0], 100, models[0], size=(len(y), 20))
# ppc_1 = pm.sample_posterior_predictive(traces[1], 100, models[1], size=(len(y), 20))
# for m_0, m_1 in zip(ppc_0['yl'].T, ppc_1['yl'].T):
#     pm.kdeplot(np.mean(m_0, 0), ax=ax, plot_kwargs={'color':'C0'})
#     pm.kdeplot(np.mean(m_1, 0), ax=ax, plot_kwargs={'color':'C1'})
# ax.plot([], label='model_0')
# ax.plot([], label='model_1')
# ax.legend(fontsize=14)
# ax.set_xlabel(u'θ', fontsize=14)
# ax.set_yticks([]);